In [27]:
import numpy as np
import os
import time
import requests
import json
import redis

In [31]:
redis_client = redis.Redis(host='redisbox', port=6379, decode_responses=True)

memory_info = redis_client.info('memory')
print(f"Current Redis memory usage: {memory_info['used_memory_human']}")
print(f"Peak memory usage: {memory_info['used_memory_peak_human']}")
print(f"Memory limit: {memory_info['maxmemory_human']}")

Current Redis memory usage: 1.97M
Peak memory usage: 2.04M
Memory limit: 2.00M


In [29]:
def print_eviction_policy():
    current_policy = redis_client.config_get('maxmemory-policy')
    print(f"Current policy: {current_policy}")

# No eviction
# redis_client.config_set('maxmemory-policy', 'noeviction')

# LRU policies
# redis_client.config_set('maxmemory-policy', 'allkeys-lru')
# redis_client.config_set('maxmemory-policy', 'volatile-lru')

# LFU policies
# redis_client.config_set('maxmemory-policy', 'allkeys-lfu')
# redis_client.config_set('maxmemory-policy', 'volatile-lfu')

# Random policies
# redis_client.config_set('maxmemory-policy', 'allkeys-random')
# redis_client.config_set('maxmemory-policy', 'volatile-random')

# TTL policy
# redis_client.config_set('maxmemory-policy', 'volatile-ttl')

Current policy: {'maxmemory-policy': 'noeviction'}
Current policy: {'maxmemory-policy': 'allkeys-lru'}
Current policy: {'maxmemory-policy': 'allkeys-random'}
Current policy: {'maxmemory-policy': 'volatile-lru'}
Current policy: {'maxmemory-policy': 'volatile-random'}
Current policy: {'maxmemory-policy': 'volatile-ttl'}


In [30]:
for i in range(10):
    key = 'test_key_' + str(i)
    try:
        write_result = redis_client.set(key, 'x' * 1024 * 100)  # 1KB of 'x' characters
        print(f"Write result for {key}: {write_result}")
    except Exception as e:
        print(f"Error writing {key} to Redis: {e}")


Error writing test_key_0 to Redis: command not allowed when used memory > 'maxmemory'.
Error writing test_key_1 to Redis: command not allowed when used memory > 'maxmemory'.
Error writing test_key_2 to Redis: command not allowed when used memory > 'maxmemory'.
Error writing test_key_3 to Redis: command not allowed when used memory > 'maxmemory'.
Error writing test_key_4 to Redis: command not allowed when used memory > 'maxmemory'.
Error writing test_key_5 to Redis: command not allowed when used memory > 'maxmemory'.
Error writing test_key_6 to Redis: command not allowed when used memory > 'maxmemory'.
Error writing test_key_7 to Redis: command not allowed when used memory > 'maxmemory'.
Error writing test_key_8 to Redis: command not allowed when used memory > 'maxmemory'.
Error writing test_key_9 to Redis: command not allowed when used memory > 'maxmemory'.


In [ ]:
result = redis_client.get('test_key')
print(f"Retrieved value: {result}")


In [34]:
def get_redis_stats(redis_client):
    # 1. Get all statistics using INFO command
    all_stats = redis_client.info()
    print("all_stats:", json.dumps(all_stats, indent=2))
    
    # Get specific sections
    memory_stats = redis_client.info('memory')
    print("memory_stats:", json.dumps(memory_stats, indent=2))

    stats = redis_client.info('stats')
    print("stats:", json.dumps(stats, indent=2))

    # 2. Key statistics
    # Total number of keys
    key_count = redis_client.dbsize()
    print("key_count:", json.dumps(key_count))

    # Memory usage of specific key
    key_memory = redis_client.memory_usage('your_key')
    print("key_memory:", json.dumps(key_memory))


    # 3. Eviction statistics from INFO
    eviction_stats = {
        'evicted_keys': stats['evicted_keys'],  # Total number of keys evicted
        'used_memory': memory_stats['used_memory'],  # Total memory used in bytes
        'used_memory_human': memory_stats['used_memory_human'],  # Human readable memory
        'maxmemory': memory_stats['maxmemory'],  # Maximum memory limit
        'maxmemory_policy': memory_stats['maxmemory_policy']  # Current eviction policy
    }
    print("eviction_stats:", json.dumps(eviction_stats, indent=2))

    # 4. Memory analysis
    # Get memory stats for all keys matching a pattern
    for key in redis_client.scan_iter("pattern*"):
        size = redis_client.memory_usage(key)
        print(f"{key}: {size} bytes")

    # 5. Get hit/miss statistics
    hits_misses = {
        'keyspace_hits': stats['keyspace_hits'],     # Number of successful lookups
        'keyspace_misses': stats['keyspace_misses'], # Number of failed lookups
        'hit_rate': stats['keyspace_hits'] / (stats['keyspace_hits'] + stats['keyspace_misses'])
    }
    print("hits_misses:", json.dumps(hits_misses, indent=2))

    # 6. Get expired/evicted counts
    expired_evicted = {
        'expired_keys': stats['expired_keys'],  # Keys that expired
        'evicted_keys': stats['evicted_keys'],  # Keys that were evicted
    }
    print("expired_evicted:", json.dumps(expired_evicted, indent=2))
    

get_redis_stats(redis_client)

all_stats: {
  "redis_version": "7.4.2",
  "redis_git_sha1": 0,
  "redis_git_dirty": 0,
  "redis_build_id": "58bcfb204733694a",
  "redis_mode": "standalone",
  "os": "Linux 5.15.167.4-microsoft-standard-WSL2 x86_64",
  "arch_bits": 64,
  "monotonic_clock": "POSIX clock_gettime",
  "multiplexing_api": "epoll",
  "atomicvar_api": "c11-builtin",
  "gcc_version": "12.2.0",
  "process_id": 1,
  "process_supervised": "no",
  "run_id": "26cfb107380a3a9d762aacce7456e95f1d84b897",
  "tcp_port": 6379,
  "server_time_usec": 1738522566364451,
  "uptime_in_seconds": 1465,
  "uptime_in_days": 0,
  "hz": 10,
  "configured_hz": 10,
  "lru_clock": 10469318,
  "executable": "/data/redis-server",
  "config_file": "/usr/local/etc/redis/redis.conf",
  "io_threads_active": 0,
  "listener0": {
    "name": "tcp",
    "bind": "-::*",
    "port": 6379
  },
  "connected_clients": 4,
  "cluster_connections": 0,
  "maxclients": 10000,
  "client_recent_max_input_buffer": 8,
  "client_recent_max_output_buffer": 0,
 